# Grid search de modelos y MLFlow

In [1]:
# Importación de librerías para los datos y gráficas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# Importación de MLFlow
import mlflow
import mlflow.sklearn
#from mlflow.models import infer_signature

# Importar elementos del modelo
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report, roc_curve


In [7]:
import dagshub
dagshub.init(repo_owner='empleo-inducido', repo_name='project-ml', mlflow=True)

Initialized MLflow to track repo "empleo-inducido/project-ml"

Repository empleo-inducido/project-ml initialized!

In [29]:
# Configuramos la MLflow Tracking URI
track_uri = "http://localhost:8080/"
mlflow.set_tracking_uri(track_uri)
# Establece la URI del registro de modelos de MLflow.
mlflow.set_registry_uri("sqlite:////tmp/registry.db")

In [30]:
# Generando el experimento o cargandolo si existe
experiment_name = "Proyecto_Dengue_AAA"
mlflow.set_experiment(experiment_name)

# Cargando la información
client = mlflow.tracking.MlflowClient()
experiment_id = client.get_experiment_by_name(experiment_name).experiment_id

# Validacion
print(f"MLflow Version: {mlflow.__version__}")
print(f"Tracking URI: {mlflow.tracking.get_tracking_uri()}")
print(f"Nombre del experimento: {experiment_name}")
print(f"ID del experimento: {experiment_id}")

MLflow Version: 2.13.0
Tracking URI: http://localhost:8080/
Nombre del experimento: Proyecto_Dengue_AAA
ID del experimento: 999845863419534997


In [6]:
mlflow.autolog()

2024/05/27 04:21:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2024/05/27 04:21:23 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


## Cargar datos

In [4]:
# Cargar datos desde el archivo Parquet
train = pd.read_parquet("../data/db/processed_train_data.parquet")

In [5]:
# Cargar datos desde el archivo Parquet
test = pd.read_parquet("../data/db/processed_test_data.parquet")

In [8]:
train.shape

(40063, 23)

In [9]:
test.shape

(10016, 23)

In [10]:
# Separar características y etiquetas
X_train = train.drop(columns=['ESTATUS_CASO'])
y_train = train['ESTATUS_CASO']

X_test = test.drop(columns=['ESTATUS_CASO'])
y_test = test['ESTATUS_CASO']

## Ejecución de modelos

In [11]:
models = {
    'LogisticRegression': {
        'model': LogisticRegression(),
        'params': {
            'C': [0.1, 1, 10, 100]
        }
    },
    'SVC': {
        'model': SVC(probability=True),
        'params': {
            'C': [0.1, 1, 10, 100],
            'kernel': ['linear', 'rbf']
        }
    },
    'RandomForestClassifier': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [10, 50, 100, 200],
            'max_depth': [None, 10, 20, 30]
        }
    },
    'GradientBoostingClassifier': {
        'model': GradientBoostingClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 5, 7]
        }
    },
    'KNeighborsClassifier': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7, 9]
        }
    }
}


## Logistic Regression

In [12]:
for modelo in models.keys():
    # Inicializar GridSearchCV
    grid_search = GridSearchCV(models[modelo]['model'], models[modelo]['params'], cv=4, scoring='accuracy')

    # Crear el nombre a utilizar para guardar el modelo
    fecha_hora_actual = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre = f"{modelo}_{fecha_hora_actual}"

    # Iniciar una nueva ejecución en MLflow con un nombre específico
    with mlflow.start_run(run_name=nombre):
        # Ajustar GridSearchCV en los datos de entrenamiento
        grid_search.fit(X_train, y_train)

        # Obtener los mejores hiperparámetros
        best_params = grid_search.best_params_
        print("Mejores hiperparámetros:", best_params)

        # Registrar los mejores hiperparámetros en MLflow
        mlflow.log_params(best_params)

        # Obtener el mejor modelo
        best_model = grid_search.best_estimator_

        # Realizar predicciones en el conjunto de prueba con el mejor modelo
        y_pred_best_model = best_model.predict(X_test)

        # Calcular y mostrar las métricas de evaluación
        accuracy = accuracy_score(y_test, y_pred_best_model)
        print(f"Accuracy del mejor modelo de {modelo}:", accuracy)

        # Registrar la métrica de precisión en MLflow
        mlflow.log_metric("accuracy", accuracy)

        # Matriz de confusión
        conf_matrix_best_model = confusion_matrix(y_test, y_pred_best_model)
        print("Confusion Matrix del mejor modelo:\n", conf_matrix_best_model)

        # Reporte de clasificación
        classification_rep_best_model = classification_report(y_test, y_pred_best_model)
        print("Classification Report del mejor modelo:\n", classification_rep_best_model)

        # Guardar el mejor modelo en MLflow
        mlflow.sklearn.log_model(best_model, f"best_{nombre}")

        # Opcional: Guardar la matriz de confusión como un artefacto
        plt.figure(figsize=(10, 7))
        sns.heatmap(conf_matrix_best_model, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix - Best {modelo}")

        # Guardar la figura
        plt.savefig("confusion_matrix_best_model.png")

        # Registrar la figura en MLflow
        mlflow.log_artifact("confusion_matrix_best_model.png")

    # Finalizar la ejecución (no es estrictamente necesario porque el contexto `with` ya lo hace)
    mlflow.end_run()

/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Mejores hiperparámetros: {'C': 100}
Accuracy del mejor modelo de LogisticRegression: 0.6587460063897763
Confusion Matrix del mejor modelo:
 [[5237  214  181]
 [ 808 1046  162]
 [1838  215  315]]
Classification Report del mejor modelo:
               precision    recall  f1-score   support

           1       0.66      0.93      0.77      5632
           2       0.71      0.52      0.60      2016
           3       0.48      0.13      0.21      2368

    accuracy                           0.66     10016
   macro avg       0.62      0.53      0.53     10016
weighted avg       0.63      0.66      0.61     10016

